<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/colon_disease_multiclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
!pip install gradio==3.14.0

In [ ]:
# Load the dataset from kaggle
od.download("https://www.kaggle.com/datasets/francismon/curated-colon-dataset-for-deep-learning")

In [ ]:
# Import necessary libraries and functions
import opendatasets as od
from tensorflow.keras.metrics import Precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from tensorflow.keras.layers import MaxPooling2D,Dense,Flatten,Conv2D,Input,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import gradio as gr
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Set up data generators with augmentation options
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator =datagen.flow_from_directory("/content/curated-colon-dataset-for-deep-learning/train"
    ,
    target_size=(224, 224),  # Adjust target size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator =datagen.flow_from_directory("/content/curated-colon-dataset-for-deep-learning/val",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
test_generator =datagen.flow_from_directory("/content/curated-colon-dataset-for-deep-learning/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Load MobileNetV1 base model without top (fully connected) layers
#base_model = MobileNet(weights='imagenet', include_top=False)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a Sequential model
model = Sequential()

# Add the MobileNetV1 base model to the Sequential model
model.add(base_model)
# Set MobileNetV1 layers as non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
model.add(GlobalAveragePooling2D())

# Add custom dense layers for classification
model.add(Dense(220, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',Precision()])
# Define a checkpoint callback to save the best
checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
# Train the model
history=model.fit(train_generator,validation_data=(validation_generator),epochs=10,batch_size=len(train_generator),callbacks=[checkpoint])

In [ ]:
# Load the best weights into your model
model.load_weights('best_weights.h5')

In [ ]:
# Evaluate on test_generator
model.evaluate(test_generator)

25/25 [==============================] - 27s 1s/step - loss: 0.1942 - accuracy: 0.9200 - precision: 0.9256


[0.19419117271900177, 0.9200000166893005, 0.9255989789962769]

In [ ]:
# Save the model as HDF5 file
model.save("colon_model.h5")

In [ ]:
import matplotlib.pyplot as plt
# `history.history` is a dictionary containing the training and validation metrics
# Plotting accuracy
plt.subplot(2, 1, 1)  # 2 rows, 1 column, plot 1 (top subplot)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting precision
plt.subplot(2, 1, 2)  # 2 rows, 1 column, plot 2 (bottom subplot)
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()  # Adjust subplot layout to avoid overlap

# Show plot
plt.show()

In [ ]:
def diagnosis(img_path):
    # Convert the numpy array to a PIL Image object
    img = Image.open(img_path)

    # Convert grayscale image to RGB
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize the image to match the model's expected input size
    img = img.resize((224, 224))

    # Normalize the image data
    img_array = np.array(img) / 255.0

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Use the model to predict
    prediction = np.argmax(model.predict(img_array))
    if prediction==0:
        return 'Normal'
    elif prediction==1:
      return "ulcreative colitis"
    elif prediction==2:
      return "Polyps"
    else:
        return 'Esophagitis'


In [ ]:
# Example usage 1
image_path="/content/curated-colon-dataset-for-deep-learning/train/0_normal/train_normal_ (10).jpg"
diagnosis(image_path)

1/1 [==============================] - 0s 421ms/step


'Normal'

In [ ]:
# Example usage 2
image_path="/content/curated-colon-dataset-for-deep-learning/val/1_ulcerative_colitis/val_ulcer_ (101).jpg"
diagnosis(image_path)

1/1 [==============================] - 0s 37ms/step


'ulcreative colitis'

In [ ]:
# Example usage 3
image_path="/content/curated-colon-dataset-for-deep-learning/test/3_esophagitis/test_esophagitis_ (100).jpg"
diagnosis(image_path)

1/1 [==============================] - 0s 65ms/step


'Esophagitis'

In [ ]:
# Define a function suitable for gradio
def gradio_fn(img_array):
    # Convert the numpy array to a PIL Image object
    img = Image.fromarray(np.uint8(img_array))

    # Convert grayscale image to RGB
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize the image to match the model's expected input size
    img = img.resize((224, 224))

    # Normalize the image data
    img_array = np.array(img) / 255.0

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Use the model to predict
    prediction = model.predict(img_array)

    # Use the model to predict
    prediction = np.argmax(model.predict(img_array))
    if prediction==0:
        return 'Normal'
    elif prediction==1:
      return "ulcreative colitis"
    elif prediction==2:
      return "Polyps"
    else:
        return 'Esophagitis'

In [ ]:
# Define the input component for the Gradio interface
inputs = gr.inputs.Image(shape=(224, 224))

# Define the output component for the Gradio interface
outputs = gr.outputs.Label()

# Create the Gradio interface
gr.Interface(gradio_fn, inputs, outputs, capture_session=True,share=True).launch(debug='True')